<a href="https://colab.research.google.com/github/Gab2697/Surface-classification-EMG/blob/main/EMG_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
import torch

from keras import layers 
from keras import models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers

from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop


from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
def open_pickle(path_pickle):
    f = open(path_pickle, 'rb')
    T = pickle.load(f)
    f.close()
    return T

def subject_wise_split(Participant,subject_wise,split=0.10,seed=42):
    np.random.seed(seed)
    if subject_wise:
        UniqParti=np.unique(Participant)
        num=np.round(UniqParti.shape[0]*split).astype('int64')
        np.random.shuffle(UniqParti)
        extract=UniqParti[0:num]
        test_index=np.array([],dtype='int64')
        for j in extract:
            test_index=np.append(test_index,np.where(Participant==j)[0])
        train_index=np.delete(np.arange(len(Participant)),test_index)
        np.random.shuffle(test_index)
        np.random.shuffle(train_index)

    else:
        I=np.arange(len(Participant)).astype('int64')
        np.random.shuffle(I)
        num=np.round(Participant.shape[0]*split).astype('int64')
        test_index=I[0:num]
        train_index=I[num:]
        extract=np.unique(Participant[test_index])
    return train_index,test_index,extract

def make_model_1D(input_size,filter_numb1,filter_numb2,filter_numb3,kernel_size,pool_size,opt):
    #Instantiating convnet
    model = models.Sequential()
    #filter size: 32, kenel of 3, input shape without the batch_size (one trial), padding= with zero padding
    model.add(layers.Conv1D(filter_numb1, kernel_size, kernel_regularizer=regularizers.l2(0.001),activation='relu', input_shape=(input_size),padding='same')) 
    model.add(layers.Dropout(0.5))
    model.add(layers.MaxPooling1D(pool_size))
    model.add(layers.Conv1D(filter_numb2, kernel_size, kernel_regularizer=regularizers.l2(0.001),activation='relu',padding='same'))

    # model.summary()

    #Adding a classifier on top of the convnet
    model.add(layers.Flatten()) 
    model.add(layers.Dense(filter_numb3, activation='relu')) 
    model.add(layers.Dense(1, activation='sigmoid')) 
    # model.summary()

    model.compile(optimizer = opt, loss='binary_crossentropy', metrics=['accuracy']) 
    return model 

In [4]:
#load pickle file
Ndata=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/All_emg/data.pkl') 

In [5]:
#Condition and surface number instead of array
for i in range(Ndata['Conditions'].shape[0]):
    Ndata['Conditions'][i]=Ndata['Conditions'][i][0]
    Ndata['Subject'][i]=Ndata['Subject'][i][0]

In [6]:
Surface=Ndata['Conditions']
Participant=Ndata['Subject']
del Ndata['Conditions']
del Ndata['Subject']

In [7]:
#create dictionary with numpy array
data={}

for i in Ndata.keys():
  data[i]=np.zeros([5256,7704])
  for j in range(Ndata[i].shape[0]):
    data[i][j,:]=Ndata[i][j].transpose()[:,0]

In [8]:
newdata={}
for i in data.keys():
    for j in range(8):
        newdata[f'{i}_{j}']=data[i][:,j*963:963+(j*963)]

In [10]:
train_index,test_index,extract=subject_wise_split(np.array(Participant),split=0.15,subject_wise=False,seed=5)
train_index,test_index=train_index.astype('int64'),test_index.astype('int64')        

In [13]:
newdata.shape

AttributeError: ignored

In [ ]:
cnnData={'X_train':{},'X_test':{},'y_train':{},'y_test':{}}

for i in newdata.keys():
  Data['X_train'][f'{i}']=newdata[train_index]
  Data['X_test'][f'{i}']=newdata[test_index]
  Data['y_train'][f'{i}']=Surface[train_index]
  Data['y_test'][f'{i}']=Surface[test_index]

In [ ]:
for i in Data['X_train'].keys():
    X_train.append(Data['X_train'][i])
    X_test.append(Data['X_test'][i])

In [ ]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [ ]:
x_train=X_train.reshape(X_train.shape[1],X_train.shape[0],1)
x_test=X_test.reshape(X_test.shape[1],X_test.shape[0],1)

In [ ]:
model=make_model_1D(x_train.shape[0],32,64,64,3,2,SGD(lr=0.01))
history= model.fit(x_train,y_train, epochs=10, batch_size=100,validation_data=(x_valid,y_valid))

Other stuff I tested

In [116]:
#removing gravel for now
Ndata_no_gravel=Ndata[Ndata['Conditions'] != 'gravel ']
print('shape of data with the gravel surface:',Ndata.shape)
print('shape of data without the gravel surface:',pd.DataFrame(Ndata_no_gravel).shape)
np.unique(Ndata_no_gravel['Conditions'])

shape of data with the gravel surface: (5256, 6)
shape of data without the gravel surface: (5216, 6)


array(['asphalt', 'grass  '], dtype=object)

In [118]:
# scikit-learn k-fold cross-validation
from numpy import array

# data sample
data = Ndata_no_gravel
# split into train test sets
n = len(pd.unique(Ndata_no_gravel['Subject'])) # number of participants
n_train= round(0.8*n) # number of participants for trianing
print('number of participants for the training set:', n_train)
n_test= n-n_train #number of participant for testing
#index of the last participant for trianing
n=Ndata_no_gravel.loc[Ndata_no_gravel['Subject'] == 'P28'].index
Ndata_train= Ndata_no_gravel.iloc[:n[-1],:] #select training set
Ndata_test= Ndata_no_gravel.iloc[n[-1]:,:] #select testing set
print('test set shape:', Ndata_test.shape)

#validation set
n_train2=round(n_train*0.8)
print('number of participants for the final training set:', n_train2)
n2=Ndata_train.loc[Ndata_train['Subject'] == 'P22'].index
Ndata_train2= Ndata_train.iloc[:n2[-1],:] #select final training set
Ndata_valid= Ndata_train.iloc[n2[-1]:,:] #select validation testing set
print('valid set shape:', Ndata_valid.shape, 'training set shape:', Ndata_train2.shape)

number of participants for the training set: 28
test set shape: (1859, 6)
number of participants for the final training set: 22
valid set shape: (965, 6) training set shape: (2392, 6)


In [119]:
#seperate x data and it's label

Surface_train=Ndata_train2['Conditions']
Participant_train=Ndata_train2['Subject']
del Ndata_train2['Conditions']
del Ndata_train2['Subject']

Surface_valid=Ndata_valid['Conditions']
Participant_valid=Ndata_valid['Subject']
del Ndata_valid['Conditions']
del Ndata_valid['Subject']

# TA=Ndata['Trignosensor1TAEMG1Volts']
# Gastroc=Ndata['Trignosensor2GastrocEMG2Volts']
# RF=Ndata['Trignosensor3RFEMG3Volts']
# BF=Ndata['Trignosensor4BFEMG4Volts']

In [120]:
x_train=Ndata_train2
y_train=Surface_train
x_valid=Ndata_valid
y_valid=Surface_valid